# SF Crime Classification Challenge

My approach for the kaggle's San Francisco Crime Classification challenge.

### Data exploration

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
training_data = pd.read_csv('/Users/user/Jupyter/datasets/SFCCC_dataset/train 2.csv')
training_data.tail(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607
878048,2003-01-06 00:01:00,FORGERY/COUNTERFEITING,"CHECKS, FORGERY (FELONY)",Monday,BAYVIEW,NONE,1800 Block of NEWCOMB AV,-122.394926,37.738212


In [3]:
testing_data = pd.read_csv('/Users/user/Jupyter/datasets/SFCCC_dataset/train 2.csv')
testing_data.tail(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607
878048,2003-01-06 00:01:00,FORGERY/COUNTERFEITING,"CHECKS, FORGERY (FELONY)",Monday,BAYVIEW,NONE,1800 Block of NEWCOMB AV,-122.394926,37.738212


In [4]:
training_data.shape, testing_data.shape

((878049, 9), (878049, 9))

In [5]:
training_data.dtypes

Dates          object
Category       object
Descript       object
DayOfWeek      object
PdDistrict     object
Resolution     object
Address        object
X             float64
Y             float64
dtype: object

In [6]:
testing_data.dtypes

Dates          object
Category       object
Descript       object
DayOfWeek      object
PdDistrict     object
Resolution     object
Address        object
X             float64
Y             float64
dtype: object

### Check missing values

In [7]:
training_data.apply(lambda x: sum(x.isnull()))

Dates         0
Category      0
Descript      0
DayOfWeek     0
PdDistrict    0
Resolution    0
Address       0
X             0
Y             0
dtype: int64

In [8]:
testing_data.apply(lambda x: sum(x.isnull()))

Dates         0
Category      0
Descript      0
DayOfWeek     0
PdDistrict    0
Resolution    0
Address       0
X             0
Y             0
dtype: int64

### Look at categories of all object variables:

In [9]:
columns = ['Dates','Category','Descript','DayOfWeek','PdDistrict','Resolution','Address','X','Y']
for column in columns:
    print ('\n+++ Frequency count for variable %s +++\n'%column)
    print (training_data[column].value_counts())


+++ Frequency count for variable Dates +++

2011-01-01 00:01:00    185
2006-01-01 00:01:00    136
2012-01-01 00:01:00     94
2006-01-01 12:00:00     63
2007-06-01 00:01:00     61
2006-06-01 00:01:00     58
2010-06-01 00:01:00     56
2010-08-01 00:01:00     55
2008-04-01 00:01:00     53
2013-11-01 00:01:00     52
2010-11-01 00:01:00     51
2008-11-01 00:01:00     51
2013-05-01 00:01:00     51
2006-07-01 00:01:00     51
2011-06-01 00:01:00     50
2005-06-01 00:01:00     50
2005-07-01 00:01:00     49
2008-06-01 00:01:00     48
2009-09-01 00:01:00     46
2013-06-01 12:00:00     46
2012-06-01 00:01:00     46
2006-11-01 00:01:00     46
2009-04-01 00:01:00     45
2009-10-01 00:01:00     45
2008-10-01 00:01:00     45
2007-07-01 00:01:00     44
2003-10-01 00:01:00     44
2013-12-01 00:01:00     44
2004-10-01 00:01:00     44
2006-12-01 00:01:00     43
                      ... 
2008-10-05 22:05:00      1
2010-11-15 23:44:00      1
2003-02-19 20:15:00      1
2014-01-07 07:05:00      1
2014-07-23

### Categorical values

#### Mining association rules
1. Create a new dataframe with only categorical values
2. Use FP growth to mine association rules

In [20]:
#Category 	Descript 	DayOfWeek 	PdDistrict 	Resolution 	Address

categorical_vals = training_data[['Category','Descript','DayOfWeek','PdDistrict','Resolution','Address']]
categorical_vals.head(2)


,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address
0,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST
1,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST


In [21]:
categorical_vals.to_csv('/Users/user/Jupyter/datasets/SFCCC_dataset/categorical_vals.csv', index=False)

In [25]:
import Orange
data = Orange.data.Table("/Users/user/Jupyter/datasets/SFCCC_dataset/categorical_vals.csv")

In [26]:
data.domain.attributes

(DiscreteVariable('Category', values=['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY', ...]),
 DiscreteVariable('Descript', values=['ABANDONMENT OF CHILD', 'ABORTION', 'ACCESS CARD INFORMATION, PUBLICATION OF', 'ACCESS CARD INFORMATION, THEFT OF', 'ACCIDENTAL BURNS', ...]),
 DiscreteVariable('DayOfWeek', values=['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', ...]),
 DiscreteVariable('PdDistrict', values=['BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', ...]),
 DiscreteVariable('Resolution', values=['ARREST, BOOKED', 'ARREST, CITED', 'CLEARED-CONTACT JUVENILE FOR MORE INFO', 'COMPLAINANT REFUSES TO PROSECUTE', 'DISTRICT ATTORNEY REFUSES TO PROSECUTE', ...]))

In [28]:
from orangecontrib.associate.fpgrowth import *  
X, mapping = OneHot.encode(data, include_class=True)

### numerical values

### Model fiting & CV